In [ ]:
'''
This notebook covers how to use tf.keras to build a classification model like what we talked about in the previous series.
'''

'\nThis notebook covers how to use tf.keras to build a classification model like what we talked about in the previous series.\n'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
from tqdm import tqdm
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.keras.backend.clear_session()  # For easy reset of notebook state.

# CHANGE THESE TO FIT YOUR FOLDER NAMES
full_folder = '/content/gdrive/My Drive/Shopee W2-4 Dataset/'
data_folder = os.path.join( full_folder, 'train_1' )
test_folder = os.path.join(full_folder, 'test')
save_model_folder = os.path.join( full_folder, 'models' )
results_folder = os.path.join(full_folder, 'results')

input_shape = (224,224,3)

In [ ]:
model_context = 'model_loose_c'

In [ ]:
# When include_top=False, we are discarding the 1000 category predictions
transfer_model = tf.keras.applications.InceptionResNetV2(input_shape=input_shape, include_top=False)

In [ ]:
# Now let's rig together a new model
def transfer_learning_model(input_shape, base_model, model_name='transfer_learning_model', num_cat = 42):
  # Freeze the base model
  for layer in base_model.layers:
    layer.trainable = False
  inputs = keras.Input(shape=input_shape)
  # First, run the input through the power model. x contains good extracted features.
  x = base_model(inputs)
  # Notice that the rest below are more or less the same
  x = layers.GlobalAveragePooling2D()(x) #2048

  x = layers.Dense(1024)(x)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)
  
  #x = layers.Dropout(0.1)(x)
  x = layers.Dense(num_cat)(x)
  predictions = layers.Softmax()(x) # predictions = layers.Sigmoid()(x)

  model = keras.Model(inputs, predictions, name=model_name)
  # Fine tuning requires a lower learning rate. The pre-trained model will be upset by the new rookie layers otherwise.
  model.compile( optimizer=tf.keras.optimizers.Adam(0.001),
                 loss=keras.losses.CategoricalCrossentropy(from_logits=False),
                 metrics=['accuracy'] )
  return model

In [ ]:
transfer_learning_model = transfer_learning_model(input_shape, transfer_model, model_name = model_context, num_cat = 42)

In [ ]:
# Set up the data generators to read from our data_folder
bs = 80 # The batch size is 32

# An object that applies transformations to the images before they are consumed by the model
# These transformations include (1) preprocessing, like rescaling or normalization (2) data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255, # divide each pixel value by 255. Each pixel is in the range 0-255, so after division it is in 0-1
        rotation_range=20, # rotate the image between -20 to +20 degrees
        width_shift_range=0.2, # translate the image left-right for 20% of the image's width
        height_shift_range=0.2, # same, for up-down and height
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)
print('Making training data generator...')
train_gen = datagen.flow_from_directory(
        data_folder,
        target_size=input_shape[:2],
        batch_size=bs,
        subset='training')
print('Making validation data generator...')
val_gen = datagen.flow_from_directory(
        data_folder,
        target_size=input_shape[:2],
        batch_size=bs,
        subset='validation')

Making training data generator...
Found 20241 images belonging to 42 classes.
Making validation data generator...
Found 5037 images belonging to 42 classes.


In [ ]:
train_gen.class_indices

{'00': 0,
 '01': 1,
 '02': 2,
 '03': 3,
 '04': 4,
 '05': 5,
 '06': 6,
 '07': 7,
 '08': 8,
 '09': 9,
 '10': 10,
 '11': 11,
 '12': 12,
 '13': 13,
 '14': 14,
 '15': 15,
 '16': 16,
 '17': 17,
 '18': 18,
 '19': 19,
 '20': 20,
 '21': 21,
 '22': 22,
 '23': 23,
 '24': 24,
 '25': 25,
 '26': 26,
 '27': 27,
 '28': 28,
 '29': 29,
 '30': 30,
 '31': 31,
 '32': 32,
 '33': 33,
 '34': 34,
 '35': 35,
 '36': 36,
 '37': 37,
 '38': 38,
 '39': 39,
 '40': 40,
 '41': 41}

In [ ]:
# Construct a reverse mapping
label_map = {v:k for k,v in train_gen.class_indices.items()}
label_map

{0: '00',
 1: '01',
 2: '02',
 3: '03',
 4: '04',
 5: '05',
 6: '06',
 7: '07',
 8: '08',
 9: '09',
 10: '10',
 11: '11',
 12: '12',
 13: '13',
 14: '14',
 15: '15',
 16: '16',
 17: '17',
 18: '18',
 19: '19',
 20: '20',
 21: '21',
 22: '22',
 23: '23',
 24: '24',
 25: '25',
 26: '26',
 27: '27',
 28: '28',
 29: '29',
 30: '30',
 31: '31',
 32: '32',
 33: '33',
 34: '34',
 35: '35',
 36: '36',
 37: '37',
 38: '38',
 39: '39',
 40: '40',
 41: '41'}

In [ ]:
# Notice that we are left with a 7x7 square of depth 2048.
# We will apply GAP to reduce this tensor to a vector of length 2048, and train a classifier at the end to distinguish between two classes
# But first, we should disable training for the ResNet50 temporarily:
for layer in transfer_model.layers:
  layer.trainable = False

In [ ]:
# Use the same callbacks, but with a different model_context
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join( save_model_folder, '{}-best_val_loss.h5'.format(model_context) ),
    save_weights_only=False,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

# If the validation loss doesn't improve for 20 epochs, stop training
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

# If the validation loss doesn't improve for 5 epochs, reduce the learning rate to 0.2 times it's previous value
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)

In [ ]:
#Load checkpoint cell
#load_model_path = os.path.join(save_model_folder, 'model_loose_c-best_val_loss.h5')
#transfer_learning_model = tf.keras.models.load_model(load_model_path)

In [ ]:
# Warm up for 10 epochs
n_epochs_warmup=5
# Followed by 40 epochs with all params trainable
n_epochs_fullblast=30

print('Warming up for {} epochs...'.format(n_epochs_warmup))
history_warmup = transfer_learning_model.fit(train_gen,
          epochs=n_epochs_warmup,
          steps_per_epoch=train_gen.n // bs,
          validation_data=val_gen,
          validation_steps=val_gen.n // bs,
          callbacks=[model_checkpoint, earlystopping, reduce_lr])

load_model_path = os.path.join(save_model_folder, '{}-best_val_loss.h5'.format(model_context) )
del transfer_learning_model
transfer_learning_model = tf.keras.models.load_model(load_model_path)


print('Done. Unfreezing all layers and training for {} more epochs...'.format(n_epochs_fullblast))
# After the warm-up, unfreeze all the layers of the base ResNet50
for layer in transfer_learning_model.get_layer('inception_resnet_v2').layers:
  layer.trainable = True

history_fullblast = transfer_learning_model.fit(train_gen,
          epochs=n_epochs_fullblast,
          steps_per_epoch=train_gen.n // bs,
          validation_data=val_gen,
          validation_steps=val_gen.n // bs,
          callbacks=[model_checkpoint, earlystopping, reduce_lr])

load_model_path = os.path.join(save_model_folder, '{}-best_val_loss.h5'.format(model_context) )
del transfer_learning_model
transfer_learning_model = tf.keras.models.load_model(load_model_path)

Warming up for 5 epochs...
Epoch 1/5
252/252 [==============================] - 12695s 50s/step - loss: 1.1292 - accuracy: 0.6938 - val_loss: 1.0301 - val_accuracy: 0.7121 - lr: 8.0000e-06
Epoch 2/5
252/252 [==============================] - 625s 2s/step - loss: 1.1272 - accuracy: 0.6918 - val_loss: 1.0272 - val_accuracy: 0.7143 - lr: 8.0000e-06
Epoch 3/5
252/252 [==============================] - 609s 2s/step - loss: 1.1232 - accuracy: 0.6902 - val_loss: 1.0123 - val_accuracy: 0.7097 - lr: 8.0000e-06
Epoch 4/5
252/252 [==============================] - 604s 2s/step - loss: 1.1147 - accuracy: 0.6925 - val_loss: 1.0353 - val_accuracy: 0.7042 - lr: 8.0000e-06
Epoch 5/5
252/252 [==============================] - 607s 2s/step - loss: 1.0959 - accuracy: 0.6963 - val_loss: 1.0106 - val_accuracy: 0.7105 - lr: 8.0000e-06
Done. Unfreezing all layers and training for 30 more epochs...
Epoch 1/30
252/252 [==============================] - 602s 2s/step - loss: 1.1139 - accuracy: 0.6944 - val_loss:

KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def run_image_on_model(img_path, model, label_map):
  pil_img = load_img(test_img_path)
  pil_img = pil_img.resize( input_shape[:2] )
  img_arr = img_to_array(pil_img)
  # Remember to normalize the image values the same way you did when you trained the model
  img_arr = img_arr / 255.
  # We need to wrap this in an np.array with dimensions (b,H,W,C). Currently, the shape is only (H,W,C)
  img_arr = np.array( [img_arr] )
  pred = model.predict(img_arr, batch_size=1)[0]
  pred_idx = np.argmax(pred)
  return label_map[pred_idx]

In [ ]:
test_df = pd.read_csv("/content/gdrive/My Drive/Shopee W2-4 Dataset/test.csv")

for i in tqdm(range(len(test_df.index))):
  test_img_path = os.path.join(test_folder, test_df.iloc[i]['filename'])
  predicted_category = run_image_on_model(test_img_path, transfer_learning_model, label_map)
  test_df.at[i, 'category'] = predicted_category
test_df["category"] = test_df["category"].apply(lambda x: "{:02}".format(x)) 
csv_path = os.path.join(results_folder, "test_{}.csv".format(model_context))
test_df.to_csv(csv_path, index = False)

100%|██████████| 12186/12186 [21:35<00:00,  9.41it/s]
